## 静态图模型的保存与加载

在静态图模型中，对应模型结构的部分为可持久化的 Program，可以保存为磁盘文件（这点不同于动态图），模型参数与动态图类似，也是用 state_dict 获取，是个状态字典，key 为参数名，value 为参数真实的值。

- 若仅需要保存/载入模型的参数用于训练调优场景，可以使用 paddle.save/paddle.load 结合静态图模型 Program 的 state_dict 达成目的。也支持保存整个模型，可以使用 paddle.save 将 Program 和state_dict 都保存下来。高层 API 兼容了动态图和静态图，因此Paddle.Model.save和Paddle.Model.load也兼容了动、静态图的保存和加载。
- 若需保存推理模型用于模型部署场景，则可以通过 paddle.static.save_inference_model、paddle.static.load_inference_model实现。

### 训练调优场景

In [1]:
import paddle
import paddle.static as static

# 开启静态图模式
paddle.enable_static()

# 创建输入数据和网络
x = paddle.static.data(name="x", shape=[None, 224], dtype='float32')
z = paddle.static.nn.fc(x, 10)

# 设置执行器开始训练
place = paddle.CPUPlace()
exe = paddle.static.Executor(place)
exe.run(paddle.static.default_startup_program())
prog = paddle.static.default_main_program()

In [2]:
# 保存模型参数
paddle.save(prog.state_dict(), '../models/static_model.pdparams')

In [3]:
# 保存模型结构
paddle.save(prog, '../models/static_model.pdmodel')

In [5]:
# 载入模型结构
# 如果只保存了模型参数，则可以跳过该步骤
prog = paddle.load('../models/static_model.pdmodel')

In [6]:
# 载入模型参数
state_dict = paddle.load('../models/static_model.pdparams')

# 将载入后的参数与模型关联
prog.set_state_dict(state_dict)

### 推理部署场景

In [7]:
import paddle
import numpy as np

# 开启静态图模式
paddle.enable_static()

# 创建输入数据和网络
startup_prog = paddle.static.default_startup_program()
main_prog = paddle.static.default_main_program()

with paddle.static.program_guard(main_prog, startup_prog):
    image = paddle.static.data(name="img", shape=[64, 784])
    w = paddle.create_parameter(shape=[784, 200], dtype='float32')
    b = paddle.create_parameter(shape=[200], dtype='float32')
    hidden_w = paddle.matmul(x=image, y=w)
    hidden_b = paddle.add(hidden_w, b)

# 设置执行器开始训练
exe = paddle.static.Executor(paddle.CPUPlace())
exe.run(startup_prog)

[]

In [8]:
""" 保存静态图推理模型

保存静态图推理模型需要指定导出路径、输入、输出变量以及执行器
paddle.static.save_inference_model 会裁剪 Program 的冗余部分，并导出两个文件： path_prefix.pdmodel、path_prefix.pdiparams
"""

path_prefix = '../models/static_infer'
paddle.static.save_inference_model(path_prefix, [image], [hidden_b], exe)

In [9]:
"""载入静态图推理模型

输入给 paddle.static.load_inference_model 的路径必须与 save_inference_model 的一致
"""

# 载入静态图推理模型
[inference_program, feed_target_names, fetch_targets] = (
    paddle.static.load_inference_model(path_prefix, exe))

tensor_img = np.array(np.random.random((64, 784)), dtype=np.float32)

results = exe.run(inference_program,
                feed={feed_target_names[0]: tensor_img},
                fetch_list=fetch_targets)

## 总结

飞桨框架同时支持动态图和静态图，优先推荐使用动态图训练，兼容支持静态图

- 训练调优场景
  - 动态图和静态图均使用 paddle.save 和 paddle.load 保存和加载模型参数
  - 或者在高层 API 训练场景下使用 paddle.Model.save 和 paddle.Model.load
- 推理部署场景
  - 动态图模型需先转为静态图模型再保存，使用 paddle.jit.save和paddle.jit.load保存和加载模型结构和参数
  - 静态图模型直接使用 paddle.static.save_inference_model和paddle.static.load_inference_model保存和加载模型结构和参数